In [ ]:
# MY SIMULATION 
import itertools
%matplotlib inline
import os
import random 
import numpy as np
from scipy.stats import sem
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import vertex_model as model
# from vertex_model.run_select_nosaveinfo import run_simulation_INM, definecolors, run_simulation_no_INM
import itertools
import numpy as np
import matplotlib.pyplot as plt
import vertex_model as model
import vertex_model.initialisation as init
import vertex_model.characterization as crt
from vertex_model.forces import TargetArea, Tension, Perimeter, Pressure
import os
import seaborn as sns
import warnings
import matplotlib.colors as colors
warnings.filterwarnings('ignore') #Don't show warnings
#from Gobal_Constant import dt, viscosity, t_G1, t_G2, t_S, A_c, J, pos_d, T1_eps, P, microns, time_hours, expansion_constant #file with necessary constants
from vertex_model.run_select import save_data
sns.set_style("whitegrid")
from scipy.spatial import Delaunay
import math

In [ ]:
#Global constants
#Global constants
dt=0.001            #time step
viscosity= 0.02  #viscosity*dv/dt = F
P= 0.0
expansion_constant = 1
Number_simulations = 50
K=1.0 #area elasticity
G= 0.04 #contractility of the cell
L=0.065 #line tensions
Lambda_0 = 0.68 #Line tensions can be reduced by increasing cell-cell adhesion or reducing actin- myosin contractility
t_end = 20
area_dimension= 177 #np.pi*(15/2.0)**2 #15 micrometros de diámetro

In [ ]:
# run simulation
def run(simulation,N_step,skip):
    N_step = math.floor(N_step)
    skip = math.floor(skip)
    iter =  itertools.islice(simulation,0,N_step,skip)
    return [cells.copy() for cells in itertools.islice(simulation,0,N_step,skip)]

#simulation without division
def basic_simulation(cells,force,dt=dt,T1_eps=0.04):
    expansion = np.array([0.0,0.0])
    while True:
#         print cells.properties['parent_group']
        cells.mesh , number_T1, edg_rem = cells.mesh.transition(T1_eps)
#         cells.properties['Gamma'][cells.mesh.face_ids == 50]=np.array([0.0])
        F = force(cells)/viscosity
#         expansion = 0.05*np.average(F*cells.mesh.vertices,1)*dt
        dv = dt*model.sum_vertices(cells.mesh.edges,F) 
#         cells.mesh = cells.mesh.moved(dv).scaled(1.0+ expansion)
        if hasattr(cells.mesh.geometry,'width'):
            expansion[0] = expansion_constant*np.average(F[0]*cells.mesh.vertices[0])*dt/(cells.mesh.geometry.width**2)
        if hasattr(cells.mesh.geometry,'height'): #Cylinder mesh doesn't have 'height' argument
            expansion[1] = np.average(F[1]*cells.mesh.vertices[1])*dt/(cells.mesh.geometry.height**2)
        cells.mesh = cells.mesh.moved(dv).scaled(1.0+expansion)
        yield cells

In [ ]:
def definecolors(cells):
    peach = '#eed5b7'
    light_blue ='#87cefa'
    pink = '#ffc0cb'
    light_green = '#98fb98'
    import matplotlib.colors as colors
    vv=sns.color_palette("hls", 10)
    v=[colors.rgb2hex(colorrgb) for colorrgb in vv]
    palette = np.array([light_green, pink,light_green,'g','r','g','m','c','',peach])
    palette = np.array([v[0],v[1],v[1], v[1],v[4],v[5],v[6],v[7],v[8],v[9],peach])
    colors = cells.properties['parent_group']
    return palette[colors]

In [ ]:
def t_extrusion(hh, ecad_cells): 
    t= 0
    cells_extrused = 0
    for s in hh: 
        m = s.mesh
        if any(cell == 0 for cell in m.area):
            cells_extrused = cells_extrused +1 
        t = t+1
        if cells_extrused == ecad_cells:
            break
    return t 
def t_extrusion2(simulation, id_ecad_cells): 
    i = 0
    t = 0
    j = 0
    c = 0
    while ((j<len(simulation)) and (i<len(id_ecad_cells))):
        
        time_step = simulation[j]
        area_vec = time_step.mesh.area
        for a in area_vec: 
            if a ==0:
                c = c+1
        if c  == len(id_ecad_cells):
            i = c
        j = j+1      
    return j
def t_extrusion3(simulation, id_ecad_cells): 
    t = 0
    for time_step in simulation:
        extrused_cells = 0
        t = t+1
        area = time_step.mesh.area
        for a in area:
            if a == 0:
                extrused_cells= extrused_cells+1
        if extrused_cells == len(id_ecad_cells): 
            break
    return t


# Main Simulation


In [ ]:
simulations = [] # to store the different simulations for the different values of noise
noise_vec = np.arange(0, 2, 0.1)
noise_vec = (0, 0.25, 0.5, 0.75)
ncellup = 18
ncellacross =  18
ncells = ncellacross*ncellup
ecad_cells_proportion = 0.2
ecad_cells = int(ecad_cells_proportion* ncells)
id_ecad_cells = random.sample(range(ncells), ecad_cells); print(id_ecad_cells)
for noise in noise_vec:     
    history_10={}
    #ids_Gamma0 = random.sample(range(1, 100), 10)
    simulations_same_noise = []
    for i in range(7):
        #run simulation with the choosen parameters
        rand =  np.random.RandomState() #random number to choose Lambda
        params = [K,G,L]  # K=x[0],G=x[1],L=x[2]
        hex_centres = init.hexagonal_centres(ncellup, ncellacross, noise, rand= np.random.RandomState())
        # create a voronoi mesh with those centres 
        mesh = init.toroidal_ic_mesh(hex_centres)
        cells = model.Cells(mesh,properties={'K':K,'Gamma':G,'P':0.0,'boundary_P':P,'Lambda':L, 'Lambda_boundary':0.5, 'A0':1.0})
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        step = int(50/dt)
        history_init_10= run(basic_simulation(cells,force),step,int(1/dt)) 
        expansion_constant = 1
        cells = history_init_10[-1].copy()
        cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
        if id_ecad_cells != []:
            cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in id_ecad_cells])[1]] = 1
        cells.properties['Gamma']= np.array([G,0])[cells.properties['parent_group']]
        cells.properties['Lambda']= np.array([L,Lambda_0])[cells.properties['parent_group']]
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        N_Step = math.floor(t_end/dt)
        history_10[i]= run(basic_simulation(cells,force),N_Step,.010/dt) 
        #save_data(i,history_10[i],'cells10')
        simulations_same_noise.append(history_10[i])

    simulations.append(simulations_same_noise)


In [ ]:
def mesh_centres(mesh):
    '''
    Compute the centres of the cells of a given mesh 
    '''
    centres_x =[]
    centres_y = [] 
    centres =np.zeros((mesh.n_face, 2))
    for i in range(mesh.n_face):       
       cell_i = cell_vertices(mesh, i) #vertices of the ith cell 
       area_i = mesh.area[i]
       if len(mesh.vertices[0][cell_i]) != 0:
        cell_center_i = centroid(mesh.vertices[0][cell_i], mesh.vertices[1][cell_i])
        centres_x.append(cell_center_i[0])
        centres_y.append(cell_center_i[1])
        #centres.append(np.array([cell_center_i[0],cell_center_i[1] ]))
        centres[i, 0] = cell_center_i[0]
        centres[i, 1] = cell_center_i[1]
    return centres_x, centres_y, centres
def cell_vertices(mesh, id):    
    cell_i = []
    for i in range(len(mesh.face_id_by_edge)):
        if mesh.face_id_by_edge[i] == id:
            cell_i.append(i)
    return cell_i
def centroid(vertexes_x, vertexes_y):   
     _len = len(vertexes_x)
     _x = sum(vertexes_x) / _len
     _y = sum(vertexes_y) / _len
     return(_x, _y)

In [ ]:
## Characterization of the final mesh of each simulation 
area = []
length_distortion = []
angle_distortion = []
edges_length = []
for i, dif_noise in enumerate(simulations):
    a_samenoise = []; edges_length_samenoise=[]; ld_samenoise = []; ad_samenoise = []
    for j in dif_noise: 
        last_mesh_j = simulations[i][0][-1].mesh
        centres_x, centres_y, centres = mesh_centres(last_mesh_j)
        tri = Delaunay(centres)
        triangles = centres[tri.simplices]
        non_obtuse, non_obtuse_indices, obtuse, obtuse_indices = crt.triangle_sieve(triangles, np.pi / 10, np.pi - (np.pi / 10))
        
        ld_samenoise.append(np.mean(crt.length_distortion(non_obtuse)))
        ad_samenoise.append(np.mean(crt.angle_distortion(non_obtuse)))
        edges_length_samenoise.append(crt.mean_perimeter(non_obtuse))
        a, std = crt.mean_area_triangles(non_obtuse)
        a_samenoise.append(a)
    
    area.append(np.mean(a_samenoise)); edges_length.append(np.mean(edges_length_samenoise))
    length_distortion.append(np.mean(ld_samenoise)); angle_distortion.append(np.mean(ad_samenoise))

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0][0].plot(noise_vec, length_distortion, label ='Length distortion')
axs[0][0].set_title('Length distortion'); axs[0][0].set_xlabel('Percentage of mutant cells')

axs[0][1].plot(noise_vec, angle_distortion, label = 'Angle distortion')
axs[0][1].set_title('Angle distortion'); axs[0][0].set_xlabel('Percentage of mutant cells')

axs[1][0].plot(noise_vec, edges_length); axs[1][0].set_title('Edges length'); axs[0][0].set_xlabel('Percentage of mutant cells')

axs[1][1].plot(noise_vec, area); axs[1][1].set_title('Area'); axs[0][0].set_xlabel('Percentage of mutant cells')





In [ ]:
extrusion_times = []
for noise in simulations:
    t_ext = 0
    for sim in noise:
        #s = sim[0] #take the simulation 
        t = t_extrusion3(sim, id_ecad_cells)
        t_ext = t_ext + t
    extrusion_times.append(t_ext/5)
plt.plot(noise_vec,extrusion_times, '.-', color='blue')
plt.ylabel('Extrusion time'); plt.xlabel('Initial mesh noise')
print(extrusion_times)


In [ ]:
extrusion_times =[] #different extrusion times for the different simulations depending on the noise 
for sim in simulations: 
    s = sim[0]
    extrusion_times.append(t_extrusion3(s, id_ecad_cells))
plt.plot(noise_vec,extrusion_times, '.-', color='blue')
plt.ylabel('Extrusion time'); plt.xlabel('Initial mesh noise')
print(extrusion_times)

In [ ]:
t = t_extrusion3(simulations[1][0], id_ecad_cells)

In [ ]:
i=0
hh = simulations[-5][0] #choose the simulation to plot 

for cells in hh:
    #ids_nofronter = [1,15,30,6,98, 23,42,56,68,71]
    #cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
    #cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in ids_nofronter])[1]] = 1
    cells.properties['color'] = definecolors(cells)
    cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
    #ids_Gamma0 = [1,15,30,6,98, 23,42,56,68,71]
    #cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in ids_Gamma0])[1]] = 1
# definition of the frame for the interactive video
history_10_plot = hh[0:40]#+history1
fig=plt.figure()
fig.set_size_inches(6,6);
ax = fig.gca()
# initialization function: plot the background of each frame
def init_fig():
    ax = plt.figure();
    return (ax,)
# animation function. This is called sequentially
def animate_fig(i):
    cells_array=history_10_plot
    v_max = np.max((np.max(cells_array[-1].mesh.vertices), np.max(cells_array[0].mesh.vertices)))
    size = 2.0*v_max
    cells= history_10_plot[i]
    return model.draw(cells,ax,size)
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig,animate_fig, init_func=init_fig,
                               frames=(len(history_10_plot)))
HTML(anim.to_html5_video())

# Area


## Area of mutant cells


In [ ]:
good_sim = []
for sim in simulations:
    s = sim[0]
    good_sim.append(s)
c = 0

In [ ]:
a = good_sim[0]
b = a[0]
len(good_sim[1][0])
good_sim[0][1]

In [ ]:
def rescale(target_area, area0, my_area): 
    ratio = target_area/area0; 
    new_area = []
    for elem in my_area: 
        new_area.append(elem*ratio)
    return new_area

In [ ]:
all_areas = [] 
errors = []
for i, noise in enumerate(simulations): #para cada distinto ruido 
    amcdifnoise = []
    area0 = []
    for j, sim in enumerate(noise): # v es una simulacion 
        #v = v[0]

        a = []
        for k, time_step in enumerate(sim): 
            cells_area = time_step.mesh.area
            cells_area_mutant = cells_area[id_ecad_cells]
            a.append(np.mean(cells_area_mutant))
            if ((i== 0) & (k==0)) : 
                area0.append(a[0])
        amcdifnoise.append(a)
        
     
    mean_area = np.mean(amcdifnoise, 0)
    ratio = np.mean(area0)
    mean_area_scaled = rescale(290,0.6793793803170991, mean_area)
    std_a = np.sem(amcdifnoise, 0)
    area_same =[]#para cada simulacion con el mismo ruido
    all_areas.append(mean_area_scaled)
    errors.append(std_a)
    plt.plot(np.arange(1,21,1), mean_area_scaled[0:20], label = f'Noise simulation = {noise_vec[i]}')
    #plt.errorbar(np.arange(1,21,1),  mean_area[0:20], yerr = std_a[0:20], capsize = 1)

    plt.legend()
plt.xlabel('Time steps'); plt.ylabel('Area of mutant cells')
noise_vec
print(all_areas[1])
print(errors[1])

In [ ]:
path=r'C:\Users\natal\OneDrive\Documentos\MASTER\TFM\Simulations\Time_extrusion' 
name_folder = 'sim_'+'%0.2f'%ecad_cells_proportion  + '3sep'

new_path = os.path.join(path, name_folder)
if not os.path.exists(new_path):
    os.makedirs(new_path)
    # last meshes folder 


file_0 = os.path.join(new_path, '0.txt')
np.savetxt(file_0, all_areas[0], delimiter=',')

file_025 = os.path.join(new_path, '0.25.txt')
np.savetxt(file_025, all_areas[1], delimiter=',')
file_05 = os.path.join(new_path, '0.5.txt')
np.savetxt(file_05, all_areas[2], delimiter=',')
file_07 = os.path.join(new_path, '0.7.txt')
np.savetxt(file_07, all_areas[3], delimiter=',')


In [ ]:
plt.plot(np.arange(1,11,1), all_areas[0][0:10], color = 'tab:blue',label = f'Noise simulation = {noise_vec[0]}')
plt.plot(np.arange(1,11,1), all_areas[1][0:10], color = 'tab:red', label = f'Noise simulation = {noise_vec[1]}')
plt.plot(np.arange(1,11,1), all_areas[2][0:10], color = 'forestgreen', label = f'Noise simulation = {noise_vec[2]}')
plt.plot(np.arange(1,11,1), all_areas[3][0:10], color = 'mediumvioletred', label = f'Noise simulation = {noise_vec[3]}')

plt.errorbar(np.arange(1,11,1),  all_areas[0][0:10], color = 'tab:blue', ecolor = 'cornflowerblue', yerr = errors[0][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[1][0:10], color = 'tab:red', ecolor = 'tomato', yerr = errors[1][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[2][0:10], color = 'forestgreen', ecolor = 'limegreen', yerr = errors[2][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[3][0:10], color = 'mediumvioletred', ecolor = 'hotpink', yerr = errors[3][0:10], capsize = 1)
plt.legend()

In [ ]:
def rescale2(target_area, area0, my_area): 
    ratio = target_area/area0; 

    n = len(my_area);
    m = len(my_area[0])
    new_area = np.zeros((n, m))
    for i in range(n):
        for j in range(m): 
            new_area[i][j] = my_area[i][j]*ratio 
  
    return new_area

In [ ]:

# good sim tiene 7 simulaciones (dif ridio) cada una con las 5 de las q tengo q hacer la media ok
all_areas = [] 
errors = []
for i, noise in enumerate(simulations): #para cada distinto ruido 
    difnoise = []
    area0 = []
    for j, sim in enumerate(noise): # v es una simulacion 
        #v = v[0]

        a = []
        for k, time_step in enumerate(sim): 
            cells_area = time_step.mesh.area
            cells_area_mutant = cells_area[id_ecad_cells]
            a.append(np.mean(cells_area_mutant))
            if ((i== 0) & (k==0)) : 
                area0.append(a[0])
        difnoise.append(a)
        
    difnoise_scaled = rescale2(290, 0.6793793803170991, difnoise) 
    mean_area = np.mean(difnoise_scaled, 0)
    #ratio = np.mean(area0)
    #mean_area_scaled = rescale(290,0.6793793803170991, mean_area)
    se = sem(difnoise_scaled, 0)
    area_same =[]#para cada simulacion con el mismo ruido
    all_areas.append(mean_area)
    errors.append(se)
    plt.plot(np.arange(1,21,1), mean_area[0:20], label = f'Noise simulation = {noise_vec[i]}')
    #plt.errorbar(np.arange(1,21,1),  mean_area[0:20], yerr = std_a[0:20], capsize = 1)

    plt.legend()
plt.xlabel('Time steps'); plt.ylabel('Area of mutant cells')
noise_vec
print(errors[1][0])

In [ ]:
plt.plot(np.arange(1,11,1), all_areas[0][0:10], color = 'tab:blue',label = f'Noise simulation = {noise_vec[0]}')
plt.plot(np.arange(1,11,1), all_areas[1][0:10], color = 'tab:red', label = f'Noise simulation = {noise_vec[1]}')
plt.plot(np.arange(1,11,1), all_areas[2][0:10], color = 'tab:green', label = f'Noise simulation = {noise_vec[2]}')
plt.plot(np.arange(1,11,1), all_areas[3][0:10], color = 'tab:orange', label = f'Noise simulation = {noise_vec[3]}')
#plt.plot(np.arange(1,11,1), all_areas[4][0:10], color = 'tab:purple', label = f'Noise simulation = {noise_vec[4]}')

plt.errorbar(np.arange(1,11,1),  all_areas[0][0:10], color = 'tab:blue', ecolor = 'cornflowerblue', yerr = errors[0][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[1][0:10], color = 'tab:red', ecolor = 'tomato', yerr = errors[1][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[2][0:10], color = 'tab:green', ecolor = 'forestgreen', yerr = errors[2][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[3][0:10], color = 'tab:orange', ecolor = 'sandybrown', yerr = errors[3][0:10], capsize = 1)
#plt.errorbar(np.arange(1,11,1),  all_areas[4][0:10], color = 'tab:purple', ecolor = 'plum', yerr = errors[4][0:10], capsize = 1)

plt.legend(); plt.xlabel('Time steps');plt.ylabel('Mutant cells area'+ r'$  \;(\mu m) $')


In [ ]:
root= r'C:\Users\natal\OneDrive\Documentos\MASTER\TFM\Simulations\Time_extrusion\sim_0.20_16_5sep'
p = os.path.join(root, '0mean.txt')

with open(p,'r') as area_file: 
        lines = area_file.readlines()
        m0 = [float(line.strip()) for line in lines]


In [ ]:
plt.plot(np.arange(0,1000,1), all_areas[0][0:1000], color = 'tab:blue',label = f'Noise simulation = {noise_vec[0]}')
plt.plot(np.arange(0,1000,1), all_areas[1][0:1000], color = 'tab:red', label = f'Noise simulation = {noise_vec[1]}')
plt.plot(np.arange(0,1000,1), all_areas[2][0:1000], color = 'tab:green', label = f'Noise simulation = {noise_vec[2]}')
plt.plot(np.arange(0,1000,1), all_areas[3][0:1000], color = 'tab:orange', label = f'Noise simulation = {noise_vec[3]}')
#plt.plot(np.arange(1,101,1), all_areas[4][0:100], color = 'tab:purple', label = f'Noise simulation = {noise_vec[4]}')
plt.legend()

In [ ]:
path=r'C:\Users\natal\OneDrive\Documentos\MASTER\TFM\Simulations\Time_extrusion' 
name_folder = 'sim_'+'%0.2f'%ecad_cells_proportion +'_18' + '_7sep'

new_path = os.path.join(path, name_folder)
if not os.path.exists(new_path):
    os.makedirs(new_path)
    # last meshes folder 


file_0m = os.path.join(new_path, '0mean.txt')
np.savetxt(file_0m, all_areas[0], delimiter=',')
file_0std = os.path.join(new_path, '0std.txt')
np.savetxt(file_0std, errors[0], delimiter=',')

file_025m = os.path.join(new_path, '0.25mean.txt')
np.savetxt(file_025m, all_areas[1], delimiter=',')
file_025std = os.path.join(new_path, '0.25std.txt')
np.savetxt(file_025std, errors[1], delimiter=',')

file_05m = os.path.join(new_path, '0.5mean.txt')
np.savetxt(file_05m, all_areas[2], delimiter=',');
file_05std = os.path.join(new_path, '0.5std.txt')
np.savetxt(file_05std, errors[2], delimiter=',')

file_07m = os.path.join(new_path, '0.7mean.txt')
np.savetxt(file_07m, all_areas[3], delimiter=',')
print(all_areas[3])
file_07std = os.path.join(new_path, '0.7std.txt')
np.savetxt(file_07std, errors[3], delimiter=',')

# file_1m = os.path.join(new_path, '1mean.txt')
# np.savetxt(file_1m, all_areas[3], delimiter=',')
# file_1std = os.path.join(new_path, '1std.txt')
# np.savetxt(file_1m, errors[3], delimiter=',')


# Neighbours


In [ ]:
# Evolution of the area of the mutant cells depending on the noise of the initial mesh 

for i,sim in enumerate(simulations):
    neighs = []
    for time_step in sim[0]: 
        cells_neighbors = np.bincount(time_step.mesh.face_id_by_edge)
        
        neighs.append(np.mean(cells_neighbors))
    plt.plot(np.arange(1,15,1), neighs[0:14], label = f'Initial mesh noise = {noise_vec[i]}')
plt.legend()
plt.title(f'Average of cell neighbors  in a {ncellup} x {ncellacross} mesh'); plt.xlabel('Time steps'); plt.ylabel('Number of cell neighbors')



## Area of the extrused cells 
(not only the mutant ones)



In [ ]:
# voy a coger la primera simualcion de cada distinto ruido
first_sims = []
for i, sim in enumerate(simulations): 
    first_sims.append(sim[0])

In [ ]:
# el numero de mutantes es igual para todas las simualciones, 
#lo que cambia es el ruido del tejido inicial
extrused_cells_allsims = []
for i, sim in enumerate(first_sims): 
    last_mesh_i = sim[-1].mesh
    extrused_cells = []
    for n_cell, a in enumerate(last_mesh_i.area):
        if a == 0: 
            extrused_cells.append(n_cell)
    extrused_cells_allsims.append(extrused_cells)



In [ ]:

# good sim tiene 7 simulaciones (dif ridio) cada una con las 5 de las q tengo q hacer la media ok
all_areas = [] 
errors = []
for i, noise in enumerate(first_sims): #para cada distinto ruido 
    difnoise = []
    area0 = []
    #for j, sim in enumerate(noise): # v es una simulacion 
        #v = v[0]

    a = []
    for k, time_step in enumerate(noise): 
        cells_area = time_step.mesh.area
        cells_area_mutant = cells_area[id_ecad_cells]
        a.append(np.mean(cells_area_mutant))
        if ((i== 0) & (k==0)) : 
            area0.append(a[0])
    difnoise.append(a)
        
    difnoise_scaled = rescale2(290, 0.6793793803170991, difnoise) 
    mean_area = np.mean(difnoise_scaled, 0)
    #ratio = np.mean(area0)
    #mean_area_scaled = rescale(290,0.6793793803170991, mean_area)
    std_a = np.std(difnoise_scaled, 0)
    area_same =[]#para cada simulacion con el mismo ruido
    all_areas.append(mean_area)
    errors.append(std_a)
    plt.plot(np.arange(1,21,1), mean_area[0:20], label = f'Noise simulation = {noise_vec[i]}')
    plt.errorbar(np.arange(1,21,1),  mean_area[0:20], yerr = std_a[0:20], capsize = 1)

    plt.legend()
plt.xlabel('Time steps'); plt.ylabel('Area of mutant cells')
noise_vec
print(errors[1][0])

In [ ]:
plt.plot(np.arange(1,11,1), all_areas[0][0:10], color = 'tab:blue',label = f'Noise simulation = {noise_vec[0]}')
plt.plot(np.arange(1,11,1), all_areas[1][0:10], color = 'tab:red', label = f'Noise simulation = {noise_vec[1]}')
plt.plot(np.arange(1,11,1), all_areas[2][0:10], color = 'tab:green', label = f'Noise simulation = {noise_vec[2]}')
plt.plot(np.arange(1,11,1), all_areas[3][0:10], color = 'tab:orange', label = f'Noise simulation = {noise_vec[3]}')
#plt.plot(np.arange(1,11,1), all_areas[4][0:10], color = 'tab:purple', label = f'Noise simulation = {noise_vec[4]}')

plt.errorbar(np.arange(1,11,1),  all_areas[0][0:10], color = 'tab:blue', ecolor = 'cornflowerblue', yerr = errors[0][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[1][0:10], color = 'tab:red', ecolor = 'tomato', yerr = errors[1][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[2][0:10], color = 'tab:green', ecolor = 'forestgreen', yerr = errors[2][0:10], capsize = 1)
plt.errorbar(np.arange(1,11,1),  all_areas[3][0:10], color = 'tab:orange', ecolor = 'sandybrown', yerr = errors[3][0:10], capsize = 1)
#plt.errorbar(np.arange(1,11,1),  all_areas[4][0:10], color = 'tab:purple', ecolor = 'plum', yerr = errors[4][0:10], capsize = 1)

plt.legend(); plt.xlabel('Time steps');plt.ylabel('Mutant cells area'+ r'$  \;(\mu m) $')
